In [1]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [2]:
from data import DATA_ROOT as DATA_PATH

In [3]:
import glob 

files = all_files = glob.glob(f"{DATA_PATH}/temp_train/*.csv")

In [4]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Distributed Word Embeddings")\
        .getOrCreate()

In [5]:
df = spark.read.options(header=True, maxColumns=22500).csv(files)

In [6]:
!pip install joblib
!pip install joblibspark

In [ ]:
import multiprocessing
from pyspark.sql import functions as F
from joblib import Parallel, delayed


threshold = 100
num_cores = multiprocessing.cpu_count() - 1


def get_columns_below_threshold(col):
    if col != "actual_words":
        col_sum = df.select(F.sum(col)).collect()[0][0]
        print(col, col_sum)
        if col_sum < threshold:
            return col
        else:
            return ""
        

columns = [get_columns_below_threshold(col) for col in df.columns]

1000000000 0.0
2 0.0
a 0.0
a people 0.0
abandon 2.0
abandoned person 0.0
abandonment 0.0
abase 0.0
abasement 0.0
abatement 0.0
abbess 0.0
abbot 0.0
abbreviate 0.0
abductor 0.0
aberrance 1.0
aberrancy 1.0
aberration 1.0
abeyance 0.0
abidance 0.0
abide 0.0
abide by 0.0
ability 1.0
abjuration 0.0
ablation 0.0
ablution 0.0
abnormalcy 0.0
abnormality 0.0
abode 0.0
about-face 0.0
abradant 0.0
abrade 0.0
abrase 0.0
abrasion 0.0
abrasive 0.0
abrasive material 0.0
abridge 0.0
abrogation 0.0
abrupt change 0.0
abscond 0.0
absence 0.0
absentee 1.0
absolve 0.0
absorb 0.0
absorbent 0.0
absorbent material 0.0
absorption 2.0
absquatulate 0.0
abstain 1.0
abstain from 0.0
abstainer 0.0
abstinence 0.0
abstract 0.0
abstract thought 1.0
abstraction 0.0
abundance 1.0
abuse 0.0
abut 0.0
academic administrator 1.0
academic degree 1.0
academic term 0.0
acanthopterygian 0.0
acarine 0.0
accelerate 0.0
acceleration 0.0
accelerator 0.0
accent 0.0
accent mark 0.0
accentuate 0.0
accept 0.0
acceptance 1.0
access 0.0


appellative 0.0
append 0.0
appendage 0.0
appetizer 0.0
appetizingness 0.0
applaud 1.0
applause 0.0
apple tree 0.0
appliance 0.0
applicant 0.0
application 0.0
application program 0.0
applications programme 0.0
applied math 0.0
applied mathematics 0.0
applied science 0.0
applied scientist 0.0
applier 0.0
apply 1.0
appoint 0.0
appointment 0.0
apportion 1.0
apportioning 1.0
apportionment 1.0
appraisal 0.0
appraise 0.0
appreciate 0.0
appreciation 0.0
apprehend 0.0
apprehension 1.0
apprehensiveness 0.0
apprise 0.0
apprize 0.0
approach 1.0
approach shot 0.0
approaching 1.0
appropriate 0.0
appropriateness 0.0
appropriation 0.0
approval 1.0
approve 1.0
approving 1.0
approximate 0.0
approximation 1.0
appurtenance 1.0
appurtenances 1.0
aptitude 0.0
aquatic bird 0.0
aquatic mammal 0.0
aquatic vertebrate 0.0
aquatics 0.0
aquilege 0.0
aquilegia 0.0
arab 0.0
arabian 0.0
arachnid 0.0
arachnoid 0.0
arboriculturist 0.0
arbutus 0.0
arcade 0.0
arcanum 0.0
archbishop 0.0
archeological remains 0.0
archimand

beast of burden 0.0
beat 4.0
beat out 1.0
beat up 2.0
beating 0.0
beatitude 0.0
beautician 0.0
beautify 0.0
beauty 0.0
becharm 0.0
becloud 0.0
become 1.0
bed 0.0
bed clothing 0.0
bed cover 0.0
bed covering 0.0
bed down 0.0
bed linen 0.0
bedchamber 0.0
bedclothes 0.0
bedcover 0.0
bedding 0.0
bedeck 0.0
bedevil 0.0
bedight 0.0
bedim 0.0
bedrock 0.0
bedroom 0.0
bedspread 0.0
bedstraw 0.0
bee 0.0
beef 0.0
beef cattle 0.0
beef up 0.0
beefsteak 0.0
beer 0.0
beet 0.0
beetle 0.0
befog 0.0
befool 0.0
befriend 0.0
befuddle 0.0
beg 1.0
begetter 0.0
beggar 0.0
begin 0.0
beginner 0.0
beginning 0.0
begrime 1.0
behave 0.0
behavior 0.0
behavior modification 0.0
behavior therapy 0.0
behaviour 0.0
behead 0.0
beholder 0.0
beholding 0.0
being 0.0
beingness 0.0
beleaguering 1.0
belgian sheep dog 0.0
belgian shepherd 0.0
belie 0.0
belief 1.0
believability 0.0
believe 0.0
believer 0.0
belittle 0.0
bell 0.0
bell tower 0.0
belligerence 0.0
belligerency 0.0
belligerent 0.0
belly 0.0
belong 0.0
belonging 0.0
bel

calendar year 0.0
calf 0.0
calibration 1.0
call 0.0
call back 0.0
call for 0.0
call forth 0.0
call in 0.0
call off 0.0
call up 0.0
calling 0.0
callosity 0.0
callus 0.0
calm 0.0
calm down 0.0
calmness 1.0
calorimeter 0.0
calosoma 0.0
caltrop 0.0
calumniate 0.0
calumny 1.0
cambodian monetary unit 0.0
camelia 0.0
camellia 0.0
camfer 0.0
camp 0.0
campaign 0.0
campaigner 0.0
campaigning 0.0
camper 0.0
camping bus 0.0
can 0.0
canaanitic 0.0
canaanitic language 0.0
canadian province 0.0
canal 0.0
canasta 0.0
cancel 0.0
cancer 0.0
candidacy 0.0
candidate 0.0
candidature 0.0
candidiasis 0.0
candle holder 0.0
candlepower unit 0.0
candlestick 0.0
candlewood 0.0
candy 2.0
cane 0.0
canid 1.0
canine 1.0
canis familiaris 2.0
cannabis 0.0
cannon 0.0
cannonball along 0.0
canoe 0.0
canonical hour 0.0
canopy 0.0
cant 0.0
cantonment 0.0
canvas 0.0
canvas tent 0.0
canvass 0.0
cap 0.0
capability 0.0
capableness 0.0
capacitance unit 0.0
capacitor 0.0
capacity 0.0
capacity measure 0.0
capacity unit 0.0
cape 0

In [ ]:
columns = list(filter(None, columns))
df = df.drop(*columns)
df.write.csv(f"{DATA_PATH}/embeddings_with_sum_of_columns_below_{threshold}.csv") ## df is an existing DataFrame object.